In [10]:

import pandas as pd
import numpy as np
import chess
import chess.engine
import chess.svg
import random
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from models.MiniDeepchess import SiameseNetwork, AutoEncoder
from tqdm import tqdm

# DATA

In [11]:
whiteWonFile = "./data/whiteWin.npy"
whiteLostFile = "./data/blackWin.npy"

In [12]:
# File Loading
whiteWonStates = np.load(whiteWonFile)
whiteLostStates = np.load(whiteLostFile)

# Custom Dataset Class
class SiameseDataset(Dataset):
    def __init__(self, whiteWonStates, whiteLostStates):
        """
        Initialize the dataset with white won and white lost states.
        
        Args:
            whiteWonStates (np.ndarray): Array of positions where white wins.
            whiteLostStates (np.ndarray): Array of positions where white loses.
        """
        self.sampleSize = min(len(whiteWonStates), len(whiteLostStates))
        # Shuffle the states initially
        self.whiteWonStates = whiteWonStates.copy()
        self.whiteLostStates = whiteLostStates.copy()
        np.random.shuffle(self.whiteWonStates)
        np.random.shuffle(self.whiteLostStates)
        # Take only the first sampleSize samples
        self.whiteWonStates = self.whiteWonStates[:self.sampleSize]
        self.whiteLostStates = self.whiteLostStates[:self.sampleSize]

    def __len__(self):
        """Return the number of samples in the dataset."""
        return self.sampleSize

    def __getitem__(self, index):
        """
        Get a sample pair and its labels.
        
        Args:
            index (int): Index of the sample.
            
        Returns:
            tuple: ([X1, X2], Y) where X1 and X2 are position tensors, and Y is the label tensor.
        """
        X1 = self.whiteWonStates[index]
        X2 = self.whiteLostStates[index]
        
        # Randomly swap with 50% probability
        if random.random() < 0.5:
            X1, X2 = X2, X1
            Y1, Y2 = 1, 0  # 1 for white lost, 0 for white won
        else:
            Y1, Y2 = 0, 1  # 0 for white won, 1 for white lost
        
        # Convert to PyTorch tensors
        X1 = torch.from_numpy(X1).float()
        X2 = torch.from_numpy(X2).float()
        Y = torch.tensor([Y1, Y2], dtype=torch.float32)
        
        return [X1, X2], Y

In [13]:
percentTrain = 0.8

whiteWin = np.load("./data/whiteWin.npy")
whiteLost = np.load("./data/blackWin.npy")

whiteWinTrain = whiteWin[:int(len(whiteWin) * percentTrain)]
whiteLostTrain = whiteLost[:int(len(whiteLost) * percentTrain)]
whiteWinTest = whiteWin[int(len(whiteWin) * percentTrain):]
whiteLostTest = whiteLost[int(len(whiteLost) * percentTrain):]

print(f"Train: {len(whiteWinTrain)} white wins, {len(whiteLostTrain)} black wins")
print(f"Test: {len(whiteWinTest)} white wins, {len(whiteLostTest)} black wins")

dataset_train = SiameseDataset(
    whiteWonStates=whiteWinTrain,
    whiteLostStates=whiteLostTrain
)
dataloader_train = DataLoader(dataset_train, batch_size=1028, shuffle=True)

dataset_test = SiameseDataset(
    whiteWonStates=whiteWinTest,
    whiteLostStates=whiteLostTest
)
dataloader_test = DataLoader(dataset_test, batch_size=1028, shuffle=True)

Train: 800000 white wins, 800000 black wins
Test: 200000 white wins, 200000 black wins


In [14]:
model = SiameseNetwork()
model.load_pretrained(
    feature_extractor_path="./checkpoints/AutoEncoder.pth",
    comparator_path="./checkpoints/Siamese.pth",
)

Pretrained model loaded from ./checkpoints/AutoEncoder.pth and ./checkpoints/Siamese.pth


In [15]:
print(model)

SiameseNetwork(
  (feature_extractor): AutoEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=773, out_features=600, bias=True)
      (1): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=600, out_features=400, bias=True)
      (4): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Linear(in_features=400, out_features=200, bias=True)
      (7): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.01)
      (9): Linear(in_features=200, out_features=100, bias=True)
      (10): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): LeakyReLU(negative_slope=0.01)
    )
    (decoder): Sequential(
      (0): Linear(in_features=100, out_features=200, bias=True)
      (1): BatchNorm1d(200, ep

In [16]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader, desc="Training", leave=False):
        optimizer.zero_grad()
        (X1, X2), Y = batch
        X1, X2, Y = X1.to(device), X2.to(device), Y.to(device)
        
        output = model(X1, X2)
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

@torch.no_grad()
def test(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0

    for batch in tqdm(dataloader, desc="Testing", leave=False):
        (X1, X2), Y = batch
        X1, X2, Y = X1.to(device), X2.to(device), Y.to(device)
        output = model(X1, X2)
        loss = criterion(output, Y)
        total_loss += loss.item()
    return total_loss / len(dataloader)

In [18]:
epochs = 1000
best_loss = float("inf")
patience = 5

for epoch in range(epochs):
    train_loss = train(model, dataloader_train, optimizer, criterion, device)
    test_loss = test(model, dataloader_test, criterion, device)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
    if epoch >= 1: 
        if test_loss < best_loss:
            best_loss = test_loss
            patience = 5
            torch.save(model.state_dict(), "./checkpoints/Siamese.pth")
            print("Model saved!")
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping!")
                break

Epoch 1/1000, Train Loss: 0.6050, Test Loss: 0.5865


Epoch 2/1000, Train Loss: 0.5625, Test Loss: 0.5824
Model saved!


KeyboardInterrupt: 